# KNN Leave-One-Out Grayscale

Robots Autónomos. Mapas Topológicos visuales

Autores:
    Alejandro Benítez López, Elena Benito Frey, Mario González Carbayo, Isidro López Dominguez, Blanca Martínez Donoso y Ángel Pavón Pérez

In [1]:
import cv2
import numpy as np
import glob
import os
import statistics as stats

directory = os.getcwd() + "/"

Se obtienen todas las rutas a las imágenes y se calculan cuantas hay:

In [2]:
# numero de imagenes totales en el dataset
total_img = 0
# lista con todas las rutas a las imagenes del dataset
all_filenames = []

for i in range(9):
    
    filenames = [img for img in glob.glob(directory + "landmarks_img/"+ str(i) + "/*.jpg")]
    
    for file in filenames:
        all_filenames.append(file)
        
    total_img += len(filenames)

all_filenames.sort()
print(f"Total de imagenes: {total_img}")

Total de imagenes: 664


Se define la función de KNN:

In [3]:
def knn(img, hist_dict):

    # se calcula el histograma en escala de grises
    hist = cv2.calcHist([img],[0],None,[256],[0,256])
    
    # se calcula el historgama normalizado
    hist = hist / (output_dimension[0] * output_dimension[1])
   
    min_distance = 1000000
    min_key = ''
    
    # busca el histograma mas parecido por la distancia euclidea
    for key, value in hist_dict.items():
        
        distance = np.linalg.norm(value-hist)
        
        if distance < min_distance:
            min_distance = distance
            min_key = key
    
    # devuelve el numero del landmark mas cercano
    return int(min_key[9])

Se calculan los histogramas y se normalizan:

In [4]:
# numero de imagenes por cada landmark
imgs_per_landmark         = [0] * 9
# numero de aciertos por cada landmark
aciertos_img_per_landmark = [0] * 9

# la dimension de todas las imagenes
output_dimension = (848, 480)

for i in range(total_img):
    
    # se guarda y se saca la imagen a comprobar
    leave_one_out_file = os.path.basename(all_filenames[i])
    leave_one_out_img  = cv2.imread(all_filenames[i], cv2.IMREAD_GRAYSCALE)
    real_landmark     = int(leave_one_out_file[9])
    
    print(f"Imagen {i} = {leave_one_out_file}")
    
    # diccionario donde se guardaran todos los histogramas
    hist_dict = {}
    
    for k in range(9):
        # se cargan las imagenes del landmark i
        landmark_filenames = [img for img in glob.glob(directory + "landmarks_img/"+ str(k) + "/*.jpg")]
        
        # quitamos el leave_one_out_file
        if all_filenames[i] in landmark_filenames:
            landmark_filenames.remove(all_filenames[i])
        
        
        landmark_filenames.sort()
        images    = [cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in landmark_filenames]
        img_names = [os.path.basename(f)                 for f in landmark_filenames]
      
        # se calcula el histograma de todas las demas imagenes
        for j, img in enumerate(images):
            
            # se calcula el histograma en escala de grises
            hist = cv2.calcHist([img],[0],None,[256],[0,256])

            # se calcula el historgama normalizado
            hist_norm = hist / (output_dimension[0] * output_dimension[1])

            # se añade al diccionario. La clave es el nombre de la imagen y el valor es su histograma normalizado 
            hist_dict[img_names[j]] = hist_norm 

    
    # se comprueba la imagen seleccionada con el knn
    landmark = knn(leave_one_out_img, hist_dict)
    
    # se suma uno a las iteraciones hechas en el landmark real
    imgs_per_landmark[real_landmark] += 1
    
    # si el landmark predecido por knn es igual al real, se suma uno en los aciertos de ese landmark
    if landmark == real_landmark:
        aciertos_img_per_landmark[real_landmark] += 1
    else:
        print(f"Fallo en {leave_one_out_file}")
    
    print(f"Porcentaje: {(i * 100) / total_img}")
        
# se saca el porcentaje parcial y total:
porcentaje_total = []

for i in range(len(imgs_per_landmark)):
    x = (aciertos_img_per_landmark[i] * 100) / imgs_per_landmark[i]
    porcentaje_total.append(x)

print("Porcentaje por cada landmark:")
print("\t" + str(porcentaje_total))
print("Porcentaje medio:")
print("\t" + str(stats.mean(porcentaje_total)))

Imagen 0 = landmark_0_frame_0.jpg
Porcentaje: 0.0
Imagen 1 = landmark_0_frame_1.jpg
Porcentaje: 0.15060240963855423
Imagen 2 = landmark_0_frame_10.jpg
Porcentaje: 0.30120481927710846
Imagen 3 = landmark_0_frame_11.jpg
Porcentaje: 0.45180722891566266
Imagen 4 = landmark_0_frame_12.jpg
Porcentaje: 0.6024096385542169
Imagen 5 = landmark_0_frame_13.jpg
Porcentaje: 0.7530120481927711
Imagen 6 = landmark_0_frame_14.jpg
Porcentaje: 0.9036144578313253
Imagen 7 = landmark_0_frame_15.jpg
Porcentaje: 1.0542168674698795
Imagen 8 = landmark_0_frame_16.jpg
Porcentaje: 1.2048192771084338
Imagen 9 = landmark_0_frame_17.jpg
Porcentaje: 1.355421686746988
Imagen 10 = landmark_0_frame_18.jpg
Porcentaje: 1.5060240963855422
Imagen 11 = landmark_0_frame_19.jpg
Porcentaje: 1.6566265060240963
Imagen 12 = landmark_0_frame_2.jpg
Porcentaje: 1.8072289156626506
Imagen 13 = landmark_0_frame_20.jpg
Porcentaje: 1.9578313253012047
Imagen 14 = landmark_0_frame_21.jpg
Porcentaje: 2.108433734939759
Imagen 15 = landmark_0

Porcentaje: 18.674698795180724
Imagen 125 = landmark_1_frame_56.jpg
Porcentaje: 18.825301204819276
Imagen 126 = landmark_1_frame_57.jpg
Porcentaje: 18.97590361445783
Imagen 127 = landmark_1_frame_58.jpg
Porcentaje: 19.126506024096386
Imagen 128 = landmark_1_frame_59.jpg
Porcentaje: 19.27710843373494
Imagen 129 = landmark_1_frame_6.jpg
Porcentaje: 19.427710843373493
Imagen 130 = landmark_1_frame_60.jpg
Porcentaje: 19.57831325301205
Imagen 131 = landmark_1_frame_61.jpg
Porcentaje: 19.728915662650603
Imagen 132 = landmark_1_frame_7.jpg
Porcentaje: 19.879518072289155
Imagen 133 = landmark_1_frame_8.jpg
Porcentaje: 20.03012048192771
Imagen 134 = landmark_1_frame_9.jpg
Porcentaje: 20.180722891566266
Imagen 135 = landmark_2_frame_0.jpg
Porcentaje: 20.33132530120482
Imagen 136 = landmark_2_frame_1.jpg
Porcentaje: 20.481927710843372
Imagen 137 = landmark_2_frame_10.jpg
Porcentaje: 20.632530120481928
Imagen 138 = landmark_2_frame_11.jpg
Porcentaje: 20.783132530120483
Imagen 139 = landmark_2_fram

Porcentaje: 36.897590361445786
Imagen 246 = landmark_3_frame_25.jpg
Porcentaje: 37.04819277108434
Imagen 247 = landmark_3_frame_26.jpg
Porcentaje: 37.19879518072289
Imagen 248 = landmark_3_frame_27.jpg
Porcentaje: 37.34939759036145
Imagen 249 = landmark_3_frame_28.jpg
Porcentaje: 37.5
Imagen 250 = landmark_3_frame_29.jpg
Porcentaje: 37.65060240963855
Imagen 251 = landmark_3_frame_3.jpg
Porcentaje: 37.80120481927711
Imagen 252 = landmark_3_frame_30.jpg
Porcentaje: 37.95180722891566
Imagen 253 = landmark_3_frame_31.jpg
Porcentaje: 38.102409638554214
Imagen 254 = landmark_3_frame_32.jpg
Porcentaje: 38.25301204819277
Imagen 255 = landmark_3_frame_33.jpg
Porcentaje: 38.403614457831324
Imagen 256 = landmark_3_frame_34.jpg
Porcentaje: 38.55421686746988
Imagen 257 = landmark_3_frame_35.jpg
Porcentaje: 38.704819277108435
Imagen 258 = landmark_3_frame_36.jpg
Porcentaje: 38.855421686746986
Imagen 259 = landmark_3_frame_37.jpg
Porcentaje: 39.006024096385545
Imagen 260 = landmark_3_frame_38.jpg
Por

Porcentaje: 55.42168674698795
Imagen 369 = landmark_4_frame_8.jpg
Porcentaje: 55.5722891566265
Imagen 370 = landmark_4_frame_9.jpg
Porcentaje: 55.72289156626506
Imagen 371 = landmark_5_frame_0.jpg
Porcentaje: 55.873493975903614
Imagen 372 = landmark_5_frame_1.jpg
Porcentaje: 56.024096385542165
Imagen 373 = landmark_5_frame_10.jpg
Porcentaje: 56.174698795180724
Imagen 374 = landmark_5_frame_11.jpg
Fallo en landmark_5_frame_11.jpg
Porcentaje: 56.325301204819276
Imagen 375 = landmark_5_frame_12.jpg
Porcentaje: 56.475903614457835
Imagen 376 = landmark_5_frame_13.jpg
Porcentaje: 56.626506024096386
Imagen 377 = landmark_5_frame_14.jpg
Porcentaje: 56.77710843373494
Imagen 378 = landmark_5_frame_15.jpg
Porcentaje: 56.9277108433735
Imagen 379 = landmark_5_frame_16.jpg
Porcentaje: 57.07831325301205
Imagen 380 = landmark_5_frame_17.jpg
Porcentaje: 57.2289156626506
Imagen 381 = landmark_5_frame_18.jpg
Porcentaje: 57.37951807228916
Imagen 382 = landmark_5_frame_19.jpg
Porcentaje: 57.53012048192771


Porcentaje: 73.79518072289157
Imagen 491 = landmark_6_frame_39.jpg
Porcentaje: 73.94578313253012
Imagen 492 = landmark_6_frame_4.jpg
Porcentaje: 74.09638554216868
Imagen 493 = landmark_6_frame_40.jpg
Porcentaje: 74.24698795180723
Imagen 494 = landmark_6_frame_41.jpg
Porcentaje: 74.39759036144578
Imagen 495 = landmark_6_frame_42.jpg
Porcentaje: 74.54819277108433
Imagen 496 = landmark_6_frame_43.jpg
Porcentaje: 74.6987951807229
Imagen 497 = landmark_6_frame_44.jpg
Porcentaje: 74.84939759036145
Imagen 498 = landmark_6_frame_45.jpg
Porcentaje: 75.0
Imagen 499 = landmark_6_frame_46.jpg
Porcentaje: 75.15060240963855
Imagen 500 = landmark_6_frame_47.jpg
Porcentaje: 75.3012048192771
Imagen 501 = landmark_6_frame_48.jpg
Porcentaje: 75.45180722891567
Imagen 502 = landmark_6_frame_49.jpg
Porcentaje: 75.60240963855422
Imagen 503 = landmark_6_frame_5.jpg
Porcentaje: 75.75301204819277
Imagen 504 = landmark_6_frame_50.jpg
Porcentaje: 75.90361445783132
Imagen 505 = landmark_6_frame_51.jpg
Porcentaje: 

Porcentaje: 92.31927710843374
Imagen 614 = landmark_8_frame_22.jpg
Porcentaje: 92.46987951807229
Imagen 615 = landmark_8_frame_23.jpg
Porcentaje: 92.62048192771084
Imagen 616 = landmark_8_frame_24.jpg
Porcentaje: 92.7710843373494
Imagen 617 = landmark_8_frame_25.jpg
Porcentaje: 92.92168674698796
Imagen 618 = landmark_8_frame_26.jpg
Porcentaje: 93.07228915662651
Imagen 619 = landmark_8_frame_27.jpg
Porcentaje: 93.22289156626506
Imagen 620 = landmark_8_frame_28.jpg
Porcentaje: 93.37349397590361
Imagen 621 = landmark_8_frame_29.jpg
Porcentaje: 93.52409638554217
Imagen 622 = landmark_8_frame_3.jpg
Porcentaje: 93.67469879518072
Imagen 623 = landmark_8_frame_30.jpg
Porcentaje: 93.82530120481928
Imagen 624 = landmark_8_frame_31.jpg
Porcentaje: 93.97590361445783
Imagen 625 = landmark_8_frame_32.jpg
Porcentaje: 94.12650602409639
Imagen 626 = landmark_8_frame_33.jpg
Porcentaje: 94.27710843373494
Imagen 627 = landmark_8_frame_34.jpg
Porcentaje: 94.42771084337349
Imagen 628 = landmark_8_frame_35.j